# The null space

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib
matplotlib.rcParams['figure.figsize'] = (12, 4)
import warnings
warnings.simplefilter("ignore")

A key principle in the problem of mapping the surfaces of stars and planets is the idea of a *null space*. The null space of a (linear) transformation is the set of input vectors that result in a zero vector as output. In the context of mapping, the null space comprises the spherical harmonics (or combinations of spherical harmonics) that do not affect the observed flux whatsoever.

A trivial example is the $Y_{1,-1}$ spherical harmonic, which does not project into the light curve for rotations about the vertical ($\hat{y}$) axis:

In [ ]:
import starry
import numpy as np

starry.config.lazy = False
starry.config.quiet = True

map = starry.Map(1)
map[1, -1] = 1
map.show(theta=np.linspace(0, 360, 50))

It is clear that as this object rotates, the total flux does not change (and in fact is exactly zero). That's because the $Y_{1,-1}$ harmonic is perfectly symmetric under such rotations.

It might be hard to think of other harmonics that behave this way, but in fact the **vast majority** of spherical harmonics are usually in the null space of the light curve problem. Let's take a deeper look at this. First, we'll load `starry` and generate a high resolution map of the Earth.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import starry

starry.config.lazy = False
starry.config.quiet = True

In [ ]:
map1 = starry.Map(30)
map1.load("earth", sigma=0.05)
map1.show(projection="rect")

Let's now compute the light curve of this map as we rotate it one full cycle about the $\hat{y}$ direction:

In [ ]:
theta = np.linspace(0, 360, 1000)
flux1 = map1.flux(theta=theta)
plt.plot(theta, flux1)
plt.xlabel(r"$\theta$ [degrees]")
plt.ylabel("Flux [arbitrary units]");

At phase 0, the prime meridian is facing the observer, and the flux drops as the Atlantic and then Pacific oceans come into view. The flux then peaks when Asia is in view, and that's the light curve of the Earth.

Now, let's create a new, identical map of the Earth:

In [ ]:
map2 = starry.Map(30)
map2.load("earth", sigma=0.05)

But this time, we'll zero out all coefficients corresponding to odd degrees above 2...

In [ ]:
for l in range(3, map2.ydeg + 1, 2):
    map2[l, :] = 0

... as well as all coefficients corresponding to negative values of `m`:

In [ ]:
for l in range(1, map2.ydeg + 1):
    map2[l, -l:0] = 0

Most of the coefficients are now zero:

In [ ]:
print(
    "Fractional size of the null space: %.3f"
    % (1 - np.count_nonzero(map2.y) / len(map2.y))
)

And here's what this silly map looks like:

In [ ]:
map2.show(projection="rect")

It doesn't really look anything like the Earth (though a sharp eye might spot the outline of Africa and some other familiar features -- barely). If you're wondering why we did this, let's plot the light curve of this new map next to the light curve of the original map of the Earth:

In [ ]:
flux2 = map2.flux(theta=theta)
plt.plot(theta, flux1, label="map1")
plt.plot(theta, flux2, "--", label="map2")
plt.legend()
plt.xlabel(r"$\theta$ [degrees]")
plt.ylabel("Flux [arbitrary units]");

Their light curves are **identical**. Even though the maps *look* completely different, the actual differences between the two maps lie entirely in the null space (by construction). This means we have no way of distinguishing between these two maps if all we have access to is the light curve.

To drive this point home, here's what the two maps look like side-by-side:

In [ ]:
from ipywidgets import widgets

out1 = widgets.Output(layout={})
out2 = widgets.Output(layout={})

with out1:
    map1.show(theta=np.linspace(0, 360, 50, endpoint=False))

with out2:
    map2.show(theta=np.linspace(0, 360, 50, endpoint=False))

widgets.HBox([out1, out2])

The total amount of flux at any given time -- equal to the brightness integrated over the entire disk -- is the same in both cases, even though the surfaces look nothing alike.

For objects rotating along an axis perpendicular to the line of sight (as in the example above), the null space consists of all of the $m < 0$ harmonics, as well as all of the harmonics of degree $l = 3, 5, 7 ...$:

In [ ]:
ydeg = 5
fig, ax = plt.subplots(ydeg + 1, 2 * ydeg + 1, figsize=(12, 6))
fig.subplots_adjust(hspace=0)
for axis in ax.flatten():
    axis.set_xticks([])
    axis.set_yticks([])
    axis.spines["top"].set_visible(False)
    axis.spines["right"].set_visible(False)
    axis.spines["bottom"].set_visible(False)
    axis.spines["left"].set_visible(False)
for l in range(ydeg + 1):
    ax[l, 0].set_ylabel(
        r"$l = %d$" % l,
        rotation="horizontal",
        labelpad=20,
        y=0.38,
        fontsize=10,
        alpha=0.5,
    )
for j, m in enumerate(range(-ydeg, ydeg + 1)):
    if m < 0:
        ax[-1, j].set_xlabel(
            r"$m {=} \mathrm{-}%d$" % -m, labelpad=10, fontsize=10, alpha=0.5
        )
    else:
        ax[-1, j].set_xlabel(r"$m = %d$" % m, labelpad=10, fontsize=10, alpha=0.5)

# Loop over the orders and degrees
map = starry.Map(ydeg=ydeg, quiet=True)
for i, l in enumerate(range(ydeg + 1)):
    for j, m in enumerate(range(-l, l + 1)):

        # Null space or not?
        if (m < 0) or (l == 3) or (l == 5):
            alpha = 0.25
        else:
            alpha = 1

        # Offset the index for centered plotting
        j += ydeg - l

        # Compute the spherical harmonic
        # with no rotation
        map.reset()
        if l > 0:
            map[l, m] = 1

        # Plot the spherical harmonic
        ax[i, j].imshow(
            map.render(),
            cmap="plasma",
            interpolation="none",
            origin="lower",
            extent=(-1, 1, -1, 1),
            alpha=alpha,
        )
        ax[i, j].set_xlim(-1.1, 1.1)
        ax[i, j].set_ylim(-1.1, 1.1)

The null space for this problem is indicated by the translucent harmonics above. (Note that in general certain linear combinations of the remaining harmonics are *also* in the null space, so the problem is actually *worse* than in the example above).